# Download HaGRID data set from kaggle

In [1]:
!pip install kaggle

## Kaggle API
1. Go to your Kaggle account page: https://www.kaggle.com/
2. Click on your profile picture in the top right and select "Account"
3. Scroll down to the "API" section
4. Click the "Create New API Token" button
5. This will automatically download a file named kaggle.json to your computer
6. Move or copy this file directly into the main (root) folder of this project

In [11]:
import os

kaggle_path = "~/.kaggle/kaggle.json"

if not os.path.exists(os.path.expanduser(kaggle_path)):
    print(f"Smth went wrong!\nkaggle.json not found on {kaggle_path}")

In [13]:
def download_hagrid_annotation(output_dir="../data/raw/hagrid"):
    """
    Download the HaGRID dataset from Kaggle and extract into output_dir
    Requires Kaggle API to be configured (kaggle.json in ~/.kaggle)
    """
    os.makedirs(output_dir, exist_ok=True)

    print(f"Downloading HaGRID dataset into {output_dir}")

    !kaggle datasets download -d kapitanov/hagrid -p {output_dir} --unzip

    if len(os.listdir(output_dir)) != 0:
        print(f"HaGRID dataset was downloaded in {output_dir}")
    else:
        print(f"Smth went wrong! \n{output_dir} is empty")

## Download dataset from Kaggle using kagglehub

Use this data set because:
* Size and diversity: >550K images, 18 static gestures, different conditions (lighting, background, poses).
* Diversity of people: many different hands, ages, genders - less risk of overfitting.
* Realistic conditions: real-life images with different backgrounds, viewing angles
* Ready-made gesture classes: (✊, ✋, ✌️, etc.) can be easily expanded for our custom idea

__!!! Downloading can take more than 10 min__

In [14]:
output_dir="../data/raw/hagrid"
download_hagrid_annotation(output_dir)

Dataset URL: https://www.kaggle.com/datasets/kapitanov/hagrid
License(s): CC-BY-SA-4.0

HaGRID dataset was downloaded in ../data/raw/hagrid



  0%|          | 0.00/338M [00:00<?, ?B/s]
 54%|#####3    | 181M/338M [00:00<00:00, 1.08GB/s]
 84%|########4 | 285M/338M [00:00<00:00, 512MB/s] 
100%|##########| 338M/338M [00:00<00:00, 517MB/s]


## Analyze data

## Preprocessing

* Resize
* Normalization

# Augmentation

### Show some examples for each class
